In [1]:
from proteins import EmbeddingProtein1D

In [2]:
from utils import open_func 
from pathlib import Path

# solo este notebook ya que corre desde una subcarpeta
import sys; sys.path.append("..") 

data_path = Path("data")

> * `open_func` deberia ser una funcion que:
>    1. reciba `data_path`
>    2. reciba el *prefijo* de tus archivos `.csv`
>        e.g. `data/stability_train.csv`, `data/<prefijo>_<kind>.csv`
>    3. Devuelva un dict que tenga como llaves los `<kind>` y como valores los `.csv` en pd.DataFrame.

>    Los `.csv` deben contener las columnas `labels` y `sequences`.



> * `data_path` debe ser Path y un directorio que contenga tus `.csv`

In [3]:
open_func(Path("data"), "stability")["train"]

,labels,sequence
0,0.37,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT
1,0.62,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK
2,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK
3,1.41,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT
4,1.11,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT
...,...,...
7705,0.80,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE
7706,0.82,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY
7707,0.66,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY
7708,1.05,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR


In [10]:
args_dict = {
    "model_name": "esm1b_t33_650M_UR50S",
    "open_func": open_func,
    "data_path": data_path,
    "gpu": True,
}

emb_stabilty = EmbeddingProtein1D(**args_dict)

#_ = emb_stabilty.generate_embeddings("stability", kind="test", save=False, bs=256)
embeddings = emb_stabilty.generate_embeddings(
    "stability", 
    kind="train",
    save=True, # guardarlo en data_path/stability_train.pkl
    bs=256
)

Using cache found in /home/roberto/.cache/torch/hub/facebookresearch_esm_master
100%|██████████| 31/31 [00:48<00:00,  1.56s/it]

Embeddings saved to data/stability_embeddings_train.pkl


In [11]:
dset = emb_stabilty.generate_datasets(
    'stability',
    kind="train",
    load_embeddings=True # cargar los embeddings desde donde se espera que esten
)

In [15]:
dset["labels"][:10], dset["sequences"][:10]

(array([ 0.37,  0.62, -0.03,  1.41,  1.11,  1.24,  1.05,  0.89,  0.88,
         1.15]),
 array([b'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT',
        b'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK',
        b'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK',
        b'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT',
        b'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG',
        b'TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY',
        b'FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT',
        b'GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS'], dtype=object))

In [16]:
dset.close()

> Como mejorarlo? 
    1. En la parte de .generate_datasets deberia ser posible pasarle el `.pkl`
    2. Generalizar la lectura y escritura de archivo c/r a los nombres.
    I.e. no depender del estilo `<prefijo>_<algo>.csv`